# GCN

In [50]:
import numpy as np
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr="add")
        self.lin = torch.nn.Linear(in_channels, out_channels)
    def forward(self, x, edge_index):
        '''
        x: [num_nodes, num_node_features]
        edge_index: [2, num_edges]
        '''
        # 添加自回环
        edge_index, _ = add_self_loops(edge_index, num_nodes = x.size(0))
        #x = self.lin(x)

        row, col = edge_index
        #pyG官方文档可能忽略了这一点, degree计算不包括self-loop
        deg = degree(col, x.size(0)) - 1
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        # feature归一化需要除以deg(i)**(-0.5)和deg(j)**(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        '''
        x_j: [num_edges, out_channels],代表利用每个edge_index关联到的node
        norm: [num_edges]
        '''
        print(norm)
        return norm.view(-1,1) * x_j

In [51]:
x = np.ones((20,2))
x = torch.from_numpy(x).float()
edge_index = torch.tensor([[0,1,1,1,1,6,7,8],
                           [1,6,7,8,0,1,1,1]], 
                           dtype = torch.int64)

print(edge_index[0,:])
print(add_self_loops(edge_index))                  
print(degree(edge_index[0,:]))
print(degree(edge_index[0,:], 20))

conv = GCN(2, 1)
x = conv(x, edge_index)

tensor([0, 1, 1, 1, 1, 6, 7, 8])
(tensor([[0, 1, 1, 1, 1, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8],
        [1, 6, 7, 8, 0, 1, 1, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8]]), None)
tensor([1., 4., 0., 0., 0., 0., 1., 1., 1.])
tensor([1., 4., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 1.0000,
        0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])
